<a href="https://colab.research.google.com/github/ditarnnt/Telecom-Predic-Churn/blob/main/PP_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##IMPORT LIBRARY

In [ ]:
from google.colab import drive
import pandas as pd #pandas
import numpy as np #numpy
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #encoding data kategorikal
from sklearn.preprocessing import StandardScaler #standardization
from sklearn.model_selection import train_test_split #train test split

##IMPORT DATASET

In [ ]:
drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/My Drive/TubesAI_folder/Dataset/Telecom_customer churn.csv',keep_default_na=False,na_values=[""])

df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,...,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.000000,132.600000,0.000000,24.000000,...,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,S,NORTHWEST/ROCKY MOUNTAIN AREA,Y,N,149.98999,2.0,2.0,WCMB,0.0,0.0,O,15.0,S,S,1.0,M,4.0,3.0,C,A,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.000000,69.000000,0.0,193.333333,0.000000,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.000000,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.000000,75.333333,0.000000,157.000000,...,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,U,CHICAGO AREA,N,N,NaN,7.0,6.0,WC,1.0,1.0,NaN,1.0,S,S,1.0,M,5.0,1.0,C,A,0.0,Z,U,U,U,U,U,Y,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,5.193333,0.000000,1.000000,...,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,S,GREAT LAKES AREA,N,N,29.98999,2.0,1.0,NA,0.0,0.0,O,7.0,S,M,2.0,M,5.0,2.0,C,A,0.0,N,U,Y,U,U,U,Y,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.000000,1.333333,0.0,3.666667,0.000000,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.000000,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.000000,3.380000,0.000000,3.666667,...,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,T,CHICAGO AREA,N,N,29.98999,1.0,1.0,NA,0.0,0.0,NaN,6.0,M,M,4.0,M,6.0,1.0,C,D,0.0,U,Y,U,U,U,U,Y,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.000000,173.476667,0.000000,90.333333,...,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,U,NEW ENGLAND AREA,Y,N,149.98999,6.0,4.0,WCMB,0.0,0.0,R,5.0,M,S,1.0,M,6.0,1.0,C,O,0.0,I,U,U,U,U,U,Y,434.0,1000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.0,0.000000,0.0,7.666667,0.0,54.666667,0.000000,3.000000,0.0,45.333333,0.000000,0.000000,0.000000,0.000000,1.666667,0.000000,94.866667,0.000000,2.016667,2.666667,1.680000,0.000000,0.000000,20.666667,0.000000,23.566667,0.000000,26.666667,...,3751,74.34,452.10,178.62,410,121,73,603.0,159.0,100.0,U,LOS ANGELES AREA,N,N,29.98999,1.0,1.0,W

drive.mount('/content/gdrive')

df = pd.read_csv('/content/gdrive/My Drive/TubesAI_folder/Telecom_customer churn.csv',keep_default_na=False,na_values=[""])

##PREPROCESSING

In [ ]:
#Menentukan fitur kategorikal yang akan di-encode atau drop

categorical_columns = ['new_cell','crclscod','asl_flag','prizm_social_one',
                       'dualband','refurb_new','hnd_webcap','dwlltype','marital',
                       'infobase','HHstatin','dwllsize','ethnic','kid0_2','kid3_5',
                       'kid6_10','kid11_15','kid16_17','creditcd','Customer_ID',
                       'area','ownrent'] #fitur-fitur yang kategorikal

drop_cat_cols = [] #placeholder untuk categorical yang bakalan didrop
enc_cat_cols = [] #placeholder untuk categorical yang bakalan lanjut di-encode

for i in range(len(categorical_columns)):
  cat_col_i = categorical_columns[i]
  value_count = df[cat_col_i].value_counts()
  #print(cat_col_i,":")
  #print(value_count)
  if (len(value_count)>3):
    drop_cat_cols.append(cat_col_i)

print('Categorical columns to drop:')
print(drop_cat_cols)

enc_cat_cols = list(set(categorical_columns) - set(drop_cat_cols))

print('Categorical columns to encode:')
print(enc_cat_cols)



Categorical columns to drop:
['crclscod', 'prizm_social_one', 'dualband', 'hnd_webcap', 'marital', 'HHstatin', 'dwllsize', 'ethnic', 'Customer_ID', 'area']
Categorical columns to encode:
['kid11_15', 'creditcd', 'asl_flag', 'ownrent', 'dwlltype', 'kid6_10', 'infobase', 'kid3_5', 'new_cell', 'kid0_2', 'refurb_new', 'kid16_17']


In [ ]:
#melakukan cleaning dan filtering fitur-fitur tidak berguna
df.dropna(axis='index',inplace=True)
for dropped_col in drop_cat_cols:
  df.drop(dropped_col,axis='columns',inplace=True)
df.drop('forgntvl',axis='columns',inplace=True) #drop forgntvl, kasus khusus


df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,...,months,uniqsubs,actvsubs,new_cell,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,refurb_new,hnd_price,phones,models,truck,rv,ownrent,lor,dwlltype,adults,infobase,income,numbcars,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays
0,23.997500,219.250000,22.5000,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,-157.250000,-18.997500,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.0,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,...,61,2,1,U,N,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,N,149.98999,2.0,2.0,0.0,0.0,O,15.0,S,1.0,M,4.0,3.0,U,U,U,U,U,Y,361.0
2,16.990000,10.250000,16.9900,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,-4.250000,0.000000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,...,58,1,1,Y,N,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,N,29.98999,2.0,1.0,0.0,0.0,O,7.0,S,2.0,M,5.0,2.0,U,Y,U,U,U,Y,1504.0
4,55.230000,570.500000,71.9800,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,38.500000,0.000000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.0,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,...,57,1,1,Y,N,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,N,149.98999,6.0,4.0,0.0,0.0,R,5.0,M,1.0,M,6.0,1.0,U,U,U,U,U,Y,434.0
9,31.662500,25.500000,29.9900,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,59.500000,4.027500,0.000000,0.0,1.000000,0.0,2.333333,0.0,6.666667,0.000000,0.000000,0.0,3.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.666667,0.0,1.866667,0.0,1.666667,...,57,2,2,N,N,391,994.00000,1458.42,1457.84,976.00,377,26.51,17.75,6.85,6,3,30,54.0,7.0,34.0,N,79.98999,2.0,2.0,0.0,0.0,O,8.0,S,2.0,M,9.0,2.0,U,U,U,U,U,Y,601.0
14,212.515000,1971.500000,84.9900,2.2275,249.500000,99.800000,99.800000,0.000,35.4975,-200.500000,-106.765000,9.000000,0.0,0.000000,0.0,43.666667,0.0,344.666667,0.000000,271.333333,0.0,296.000000,0.000000,0.333333,3.333333,3.316667,126.333333,0.0,581.296667,0.000000,718.070000,49.333333,35.163333,4.666667,9.263333,351.666667,0.0,864.173333,0.0,128.666667,...,59,5,1,Y,N,33184,69161.08333,6616.72,6572.70,69104.08,33139,115.31,1212.35,581.39,2038,637,248,1817.0,576.0,182.0,N,149.98999,10.0,6.0,0.0,0.0,O,11.0,S,3.0,M,3.0,2.0,U,U,Y,U,U,Y,199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99939,49.362500,187.750000,40.0725,2.9700,18.000000,6.320000,6.125000,0.195,0.0000,-187.750000,-9.372500,1.000000,0.0,0.666667,0.0,25.333333,0.0,103.333333,0.333333,55.000000,0.0,76.666667,0.333333,0.000000,0.000000,0.000000,29.333333,0.0,76.190000,0.356667,89.133333,29.0000

In [ ]:
#menggeser fitur-fitur kategorikal ke sebelah kanan dataset agar mudah

label = ['churn'] #label
num_cols = df.columns.to_list()

def diff(first, second):
  second = set(second)
  return [item for item in first if item not in second]

num_cols = diff(num_cols,label)
num_cols = diff(num_cols,enc_cat_cols) #num_cols adalah list fitur2 numerik

cols = num_cols + enc_cat_cols + label #list kolom-kolom yang ordered, termasuk di dalamnya label
print(cols)

df = df[cols] #dataframe yang udah diorder

features = diff(cols,label) #fitur-fitur

df

['rev_Mean', 'mou_Mean', 'totmrc_Mean', 'da_Mean', 'ovrmou_Mean', 'ovrrev_Mean', 'vceovr_Mean', 'datovr_Mean', 'roam_Mean', 'change_mou', 'change_rev', 'drop_vce_Mean', 'drop_dat_Mean', 'blck_vce_Mean', 'blck_dat_Mean', 'unan_vce_Mean', 'unan_dat_Mean', 'plcd_vce_Mean', 'plcd_dat_Mean', 'recv_vce_Mean', 'recv_sms_Mean', 'comp_vce_Mean', 'comp_dat_Mean', 'custcare_Mean', 'ccrndmou_Mean', 'cc_mou_Mean', 'inonemin_Mean', 'threeway_Mean', 'mou_cvce_Mean', 'mou_cdat_Mean', 'mou_rvce_Mean', 'owylis_vce_Mean', 'mouowylisv_Mean', 'iwylis_vce_Mean', 'mouiwylisv_Mean', 'peak_vce_Mean', 'peak_dat_Mean', 'mou_peav_Mean', 'mou_pead_Mean', 'opk_vce_Mean', 'opk_dat_Mean', 'mou_opkv_Mean', 'mou_opkd_Mean', 'drop_blk_Mean', 'attempt_Mean', 'complete_Mean', 'callfwdv_Mean', 'callwait_Mean', 'months', 'uniqsubs', 'actvsubs', 'totcalls', 'totmou', 'totrev', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'avg6mou', 'avg6qty', 'avg6rev', 'hnd_price', 'phones', '

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,...,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,eqpdays,kid11_15,creditcd,asl_flag,ownrent,dwlltype,kid6_10,infobase,kid3_5,new_cell,kid0_2,refurb_new,kid16_17,churn
0,23.997500,219.250000,22.5000,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,-157.250000,-18.997500,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.0,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,...,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.98999,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,361.0,U,Y,N,O,S,U,M,U,U,U,N,U,1
2,16.990000,10.250000,16.9900,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,-4.250000,0.000000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,...,1,1,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,29.98999,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,1504.0,U,Y,N,O,S,U,M,Y,Y,U,N,U,1
4,55.230000,570.500000,71.9800,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,38.500000,0.000000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.0,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,...,1,1,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,149.98999,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,434.0,U,Y,N,R,M,U,M,U,Y,U,N,U,0
9,31.662500,25.500000,29.9900,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,59.500000,4.027500,0.000000,0.0,1.000000,0.0,2.333333,0.0,6.666667,0.000000,0.000000,0.0,3.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.666667,0.0,1.866667,0.0,1.666667,...,2,2,391,994.00000,1458.42,1457.84,976.00,377,26.51,17.75,6.85,6,3,30,54.0,7.0,34.0,79.98999,2.0,2.0,0.0,0.0,8.0,2.0,9.0,2.0,601.0,U,Y,N,O,S,U,M,U,N,U,N,U,0
14,212.515000,1971.500000,84.9900,2.2275,249.500000,99.800000,99.800000,0.000,35.4975,-200.500000,-106.765000,9.000000,0.0,0.000000,0.0,43.666667,0.0,344.666667,0.000000,271.333333,0.0,296.000000,0.000000,0.333333,3.333333,3.316667,126.333333,0.0,581.296667,0.000000,718.070000,49.333333,35.163333,4.666667,9.263333,351.666667,0.0,864.173333,0.0,128.666667,...,5,1,33184,69161.08333,6616.72,6572.70,69104.08,33139,115.31,1212.35,581.39,2038,637,248,1817.0,576.0,182.0,149.98999,10.0,6.0,0.0,0.0,11.0,3.0,3.0,2.0,199.0,U,Y,N,O,S,Y,M,U,Y,U,N,U,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99939,49.362500,187.750000,40.0725,2.9700,18.000000,6.320000,6.125000,0.195,0.0000,-187.750000,-9.372500,1.000000,0.0,0.666667,0.0,25.333333,0.0,103.333333,0.333333,55.000000,0.0,76.666667,0.333333,0.000000,0.000000,0.000000,29.333333,0.0,76.190000,0.356667,89.133333,29.000000,19.

In [ ]:
#melakukan encoding feature-feature kategorikal
for i in range(-1*len(enc_cat_cols)-1,-1):
  labelencoder = LabelEncoder() #object label encoder
  #print(i)
  df.iloc[:,i] = labelencoder.fit_transform(df.iloc[:,i]) #encode label
 
df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,...,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,eqpdays,kid11_15,creditcd,asl_flag,ownrent,dwlltype,kid6_10,infobase,kid3_5,new_cell,kid0_2,refurb_new,kid16_17,churn
0,23.997500,219.250000,22.5000,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,-157.250000,-18.997500,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.000000,42.333333,0.0,45.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.0,90.643333,0.000000,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,...,2,1,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,149.98999,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,361.0,0,1,0,0,1,0,0,0,1,0,0,0,1
2,16.990000,10.250000,16.9900,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,-4.250000,0.000000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.000000,0.333333,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,5.426667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,...,1,1,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,29.98999,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,1504.0,0,1,0,0,1,0,0,1,2,0,0,0,1
4,55.230000,570.500000,71.9800,0.0000,0.000000,0.000000,0.000000,0.000,0.0000,38.500000,0.000000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.000000,94.666667,0.0,137.000000,0.000000,8.666667,15.000000,11.076667,66.000000,0.0,285.233333,0.000000,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,...,1,1,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,149.98999,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,434.0,0,1,0,1,0,0,0,0,2,0,0,0,0
9,31.662500,25.500000,29.9900,0.2475,0.000000,0.000000,0.000000,0.000,0.0000,59.500000,4.027500,0.000000,0.0,1.000000,0.0,2.333333,0.0,6.666667,0.000000,0.000000,0.0,3.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.666667,0.0,1.866667,0.0,1.666667,...,2,2,391,994.00000,1458.42,1457.84,976.00,377,26.51,17.75,6.85,6,3,30,54.0,7.0,34.0,79.98999,2.0,2.0,0.0,0.0,8.0,2.0,9.0,2.0,601.0,0,1,0,0,1,0,0,0,0,0,0,0,0
14,212.515000,1971.500000,84.9900,2.2275,249.500000,99.800000,99.800000,0.000,35.4975,-200.500000,-106.765000,9.000000,0.0,0.000000,0.0,43.666667,0.0,344.666667,0.000000,271.333333,0.0,296.000000,0.000000,0.333333,3.333333,3.316667,126.333333,0.0,581.296667,0.000000,718.070000,49.333333,35.163333,4.666667,9.263333,351.666667,0.0,864.173333,0.0,128.666667,...,5,1,33184,69161.08333,6616.72,6572.70,69104.08,33139,115.31,1212.35,581.39,2038,637,248,1817.0,576.0,182.0,149.98999,10.0,6.0,0.0,0.0,11.0,3.0,3.0,2.0,199.0,0,1,0,0,1,1,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99939,49.362500,187.750000,40.0725,2.9700,18.000000,6.320000,6.125000,0.195,0.0000,-187.750000,-9.372500,1.000000,0.0,0.666667,0.0,25.333333,0.0,103.333333,0.333333,55.000000,0.0,76.666667,0.333333,0.000000,0.000000,0.000000,29.333333,0.0,76.190000,0.356667,89.133333,29.000000,19.

In [ ]:
#Split jadi feature dan label

X = np.array(df.drop('churn',axis='columns',inplace=False).values)
Y = np.array(df.loc[:,'churn'].values)

print(X)
print(Y)

[[ 23.9975     219.25        22.5        ...   0.           0.
    0.        ]
 [ 16.99        10.25        16.99       ...   0.           0.
    0.        ]
 [ 55.23       570.5         71.98       ...   0.           0.
    0.        ]
 ...
 [ 41.84       351.          30.         ...   0.           0.
    0.        ]
 [ 38.34       319.5         30.         ...   0.           0.
    0.        ]
 [ 31.92333333  63.33333333  16.99       ...   0.           0.
    0.        ]]
[1 1 0 ... 0 1 0]


In [ ]:
#standardisasi fitur-fitur numerik
#ada yang bilang ini ga perlu kalo pake Random Forest, jadi bisa diskip!
#saran: bikin dua dataset aja, yg distandardized sama yg engga

#X_num = X[:,:len(num_cols)]
#sc_X = StandardScaler() #object scaler
#X_num = sc_X.fit_transform(X_num)

#X[:,:len(num_cols)] = X_num
#print(X)
#print(X.shape)

In [ ]:
#INI KALAU OUTPUT CHURN HARUS 'Y' ATAU 'N' BUKAN '1' DAN '0'
#kalau engga dicomment aja

def mapchurn(x):
  if (x==1):
    return 'Y'
  else:
    return 'N'

Y_temp = np.empty(Y.shape,dtype=str)
for i in range(len(Y)):
  Y_temp[i] = mapchurn(Y[i])

print(Y)
print(Y_temp)
Y = Y_temp #Label churn sekarang berupa 'Y' atau 'N'
Y

[1 1 0 ... 0 1 0]
['Y' 'Y' 'N' ... 'N' 'Y' 'N']


array(['Y', 'Y', 'N', ..., 'N', 'Y', 'N'], dtype='<U1')

In [ ]:
#PREPROCESSING SELESAI
#Penyesuaian dimensi:

X = X.T
Y = Y.reshape(1,-1)

#DESKRIPSI:
#X adalah matriks fitur dataset dengan size FxM
#Baris F adalah fitur-fitur dataset sejumlah 89 fitur
#Kolom M adalah sample-sample dari dataset

#Y adalah array label setiap sample dengan size 1xM
#M adalah sample-sample pada dataset

print(X)
print(X.shape)
print(Y)
print(Y.shape)

[[ 23.9975      16.99        55.23       ...  41.84        38.34
   31.92333333]
 [219.25        10.25       570.5        ... 351.         319.5
   63.33333333]
 [ 22.5         16.99        71.98       ...  30.          30.
   16.99      ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
(88, 30651)
[['Y' 'Y' 'N' ... 'N' 'Y' 'N']]
(1, 30651)


##EXPORT DATASET IN CSV FORM

In [ ]:
df_preprocessed = pd.DataFrame(np.append(X.T,Y.T,axis=1),columns=cols)
df_preprocessed #dataset yang sudah dipreprocess

#NOTE!!
#df_preprocessed ini BEDA DIMENSI dengan matriks X dan vector Y sebelumnya!
#Konvensi dataframe csv ini sesuai dengan csv sebelumnya!

#INI DI-UNCOMMENT KALO MAU NGESAVE AJA DI DRIVE:
#PERHATIIN PENAMAAN SEBELUM EXPORT KE DRIVE
#df_preprocessed.to_csv('/content/gdrive/My Drive/TubesAI_folder/df_preprocessed.csv',index=False) #export ke folder TubesAI_folder di drive

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,...,uniqsubs,actvsubs,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,hnd_price,phones,models,truck,rv,lor,adults,income,numbcars,eqpdays,kid11_15,creditcd,asl_flag,ownrent,dwlltype,kid6_10,infobase,kid3_5,new_cell,kid0_2,refurb_new,kid16_17,churn
0,23.9975,219.25,22.5,0.2475,0.0,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666666667,0.0,0.666666667,0.0,6.333333333,0.0,52.33333333,0.0,42.33333333,0.0,45.0,0.0,0.0,0.0,0.0,18.0,0.0,90.64333333,0.0,97.17666667,0.0,0.0,0.0,0.0,58.0,0.0,132.6,0.0,24.0,...,2.0,1.0,1652.0,4228.0,1504.62,1453.44,4085.0,1602.0,29.66,83.37,32.69,272.0,116.0,30.0,322.0,136.0,38.0,149.98999019999997,2.0,2.0,0.0,0.0,15.0,1.0,4.0,3.0,361.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Y
1,16.99,10.25,16.99,0.0,0.0,0.0,0.0,0.0,0.0,-4.25,0.0,0.333333333,0.0,0.0,0.0,2.666666667,0.0,9.0,0.0,0.333333333,0.0,6.0,0.0,0.0,0.0,0.0,0.333333333,0.0,5.426666667,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.193333333,0.0,1.0,...,1.0,1.0,7903.0,24385.053330000002,2155.91,1934.47,24303.05,7888.0,34.54,433.98,140.86,12.0,7.0,17.0,11.0,6.0,17.0,29.98999023,2.0,1.0,0.0,0.0,7.0,2.0,5.0,2.0,1504.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,Y
2,55.23,570.5,71.98,0.0,0.0,0.0,0.0,0.0,0.0,38.5,0.0,9.666666667000001,0.0,0.666666667,0.0,77.0,0.0,222.33333330000002,0.0,94.66666667,0.0,137.0,0.0,8.666666667000001,15.0,11.07666667,66.0,0.0,285.2333333,0.0,106.33,14.66666667,10.81666667,0.666666667,0.36666666700000006,97.33333333,0.0,173.47666669999998,0.0,90.33333333,...,1.0,1.0,4485.0,14028.0,2181.12,2166.48,13965.0,4452.0,38.69,249.38,79.5,558.0,191.0,55.0,586.0,196.0,80.0,149.98999019999997,6.0,4.0,0.0,0.0,5.0,1.0,6.0,1.0,434.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,N
3,31.6625,25.5,29.99,0.2475,0.0,0.0,0.0,0.0,0.0,59.5,4.0275,0.0,0.0,1.0,0.0,2.333333333,0.0,6.666666667,0.0,0.0,0.0,3.333333333,0.0,0.0,0.0,0.0,0.0,0.0,3.8,0.0,0.0,0.0,0.0,0.0,0.0,1.6666666669999999,0.0,1.8666666669999998,0.0,1.6666666669999999,...,2.0,2.0,391.0,994.0,1458.42,1457.84,976.0,377.0,26.51,17.75,6.85,6.0,3.0,30.0,54.0,7.0,34.0,79.98999023,2.0,2.0,0.0,0.0,8.0,2.0,9.0,2.0,601.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
4,212.515,1971.5,84.99,2.2275,249.5,99.8,99.8,0.0,35.4975,-200.5,-106.765,9.0,0.0,0.0,0.0,43.66666667,0.0,344.6666667,0.0,271.3333333,0.0,296.0,0.0,0.333333333,3.333333333,3.316666667,126.33333329999999,0.0,581.2966667000001,0.0,718.07,49.33333333,35.16333333,4.666666667,9.263333333,351.6666667,0.0,864.1733332999999,0.0,128.6666667,...,5.0,1.0,33184.0,69161.08333,6616.72,6572.7,69104.08,33139.0,115.31,1212.35,581.39,2038.0,637.0,248.0,1817.0,576.0,182.0,149.98999019999997,10.0,6.0,0.0,0.0,11.0,3.0,3.0,2.0,199.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30646,49.3625,187.75,40.0725,2.97,18.0,6.32,6.125,0.195,0.0,-187.75,-9.3725,1.0,0.0,0.666666667,0.0,25.33333333,0.0,103.33333329999999,0.333333333,55.0,0.0,76.66666667,0.333333333,0.0,0.0,0.0,29.33333333,0.0,76.19,0.35666666700000005,89.13333333,29.0,19.51333333,15.66666667,15.98,73.33333333,0.0,106.23,0.0,50.0,...,1.0,1.0,3378.0,6356.0,1563.62,1505.24,6235.0,3310.0,60.21,249.4,132

##Feature Selection Weka Toolkit

In [ ]:
'''
=== Attribute Selection on all input data ===

Search Method:
	Attribute ranking.

Attribute Evaluator (supervised, Class (nominal): 89 churn):
	Information Gain Ranking Filter

Ranked attributes:
 0.025533    49 months
 0.023637    76 eqpdays
 0.011215    67 hnd_price
 0.007733     3 totmrc_Mean
 0.007376    10 change_mou
 0.005901     2 mou_Mean
 0.004948    61 avg3mou
 0.003985    62 avg3qty
 0.00289     29 mou_cvce_Mean
 0.0028      55 adjrev
 0.00246     54 totrev
 0.002374    42 mou_opkv_Mean
 0.002162    84 asl_flag
 0.00215     35 mouiwylisv_Mean
 0.002046    52 totcalls
 0.001923    64 avg6mou
 0.001915    57 adjqty
 0.001886    34 iwylis_vce_Mean
 0.00186     24 custcare_Mean
 0.001818    38 mou_peav_Mean
 0.001719    72 lor
 0.001549    11 change_rev
 0.00153     26 cc_mou_Mean
 0.001464    40 opk_vce_Mean
 0.001451    25 ccrndmou_Mean
 0.001421     6 ovrrev_Mean
 0.001402     7 vceovr_Mean
 0.00139     33 mouowylisv_Mean
 0.001386    65 avg6qty
 0.001376    69 models
 0.001329     5 ovrmou_Mean
 0.00127     32 owylis_vce_Mean
 0.001193    45 attempt_Mean
 0.001189    22 comp_vce_Mean
 0.001172    18 plcd_vce_Mean
 0.001158    46 complete_Mean
 0.001136    68 phones
 0.001126    36 peak_vce_Mean
 0.001053    86 refurb_new
 0.000879    56 adjmou
 0.00087     53 totmou
 0.00079     16 unan_vce_Mean
 0.000743    31 mou_rvce_Mean
 0.000686    50 uniqsubs
 0.000653     1 rev_Mean
 0.00062     58 avgrev
 0.00062     63 avg3rev
 0.000577    28 threeway_Mean
 0.000545     9 roam_Mean
 0.00048     44 drop_blk_Mean
 0.000282    80 kid0_2
 0.000272    74 income
 0.000234    87 kid3_5
 0           39 mou_pead_Mean
 0           13 drop_dat_Mean
 0           12 drop_vce_Mean
 0           14 blck_vce_Mean
 0           78 kid6_10
 0           15 blck_dat_Mean
 0           75 numbcars
 0           77 new_cell
 0            8 datovr_Mean
 0           79 infobase
 0           83 dwlltype
 0           85 kid16_17
 0            4 da_Mean
 0           81 kid11_15
 0           82 creditcd
 0           17 unan_dat_Mean
 0           19 plcd_dat_Mean
 0           37 peak_dat_Mean
 0           48 callwait_Mean
 0           30 mou_cdat_Mean
 0           51 actvsubs
 0           47 callfwdv_Mean
 0           20 recv_vce_Mean
 0           43 mou_opkd_Mean
 0           41 opk_dat_Mean
 0           59 avgmou
 0           60 avgqty
 0           27 inonemin_Mean
 0           66 avg6rev
 0           21 recv_sms_Mean
 0           73 adults
 0           23 comp_dat_Mean
 0           71 rv
 0           70 truck
 0           88 ownrent

Selected attributes: 49,76,67,3,10,2,61,62,29,55,54,42,84,35,52,64,57,34,24,38,72,11,26,40,25,6,7,33,65,69,5,32,45,22,18,46,68,36,86,56,53,16,31,50,1,58,63,28,9,44,80,74,87,39,13,12,14,78,15,75,77,8,79,83,85,4,81,82,17,19,37,48,30,51,47,20,43,41,59,60,27,66,21,73,23,71,70,88 : 88

'''

'\n=== Attribute Selection on all input data ===\n\nSearch Method:\n\tAttribute ranking.\n\nAttribute Evaluator (supervised, Class (nominal): 89 churn):\n\tInformation Gain Ranking Filter\n\nRanked attributes:\n 0.025533    49 months\n 0.023637    76 eqpdays\n 0.011215    67 hnd_price\n 0.007733     3 totmrc_Mean\n 0.007376    10 change_mou\n 0.005901     2 mou_Mean\n 0.004948    61 avg3mou\n 0.003985    62 avg3qty\n 0.00289     29 mou_cvce_Mean\n 0.0028      55 adjrev\n 0.00246     54 totrev\n 0.002374    42 mou_opkv_Mean\n 0.002162    84 asl_flag\n 0.00215     35 mouiwylisv_Mean\n 0.002046    52 totcalls\n 0.001923    64 avg6mou\n 0.001915    57 adjqty\n 0.001886    34 iwylis_vce_Mean\n 0.00186     24 custcare_Mean\n 0.001818    38 mou_peav_Mean\n 0.001719    72 lor\n 0.001549    11 change_rev\n 0.00153     26 cc_mou_Mean\n 0.001464    40 opk_vce_Mean\n 0.001451    25 ccrndmou_Mean\n 0.001421     6 ovrrev_Mean\n 0.001402     7 vceovr_Mean\n 0.00139     33 mouowylisv_Mean\n 0.001386  